In [3]:
import numpy as np
import os
import pandas as pd
import torch

from PIL import Image, ImageFile
from torch.utils.data import Dataset, DataLoader

# utils.ipynb 파일에서 해당 함수를 불러오기 위한 라이브러리
# jupyter notebook에서 가져올 수 있는 방법 알아보기.

'''
from utils import (
    cells_to_bboxes,
    iou_width_height as iou,
    non_max_suppression as nms,
    plot_image
)
'''

'\nfrom utils import (\n    cells_to_bboxes,\n    iou_width_height as iou,\n    non_max_suppression as nms,\n    plot_image\n)\n'

In [13]:
S = [13, 26, 52]
targets = [torch.zeros(((3, 3, 2), S, S, 6)) for S in S]

NameError: name 'torch' is not defined

## 기본 변수 정의

In [11]:
ori_dir = './train/'
img_dir = ori_dir + 'image/'
label_dir = ori_dir + 'label/'

train_csv = ori_dir + 'train.csv'

img_size = 416
S = [13, 26, 52]
C = 4

# Note these have been rescaled to be between [0, 1]
# anchor list,ipynb 파일에서 값을 조정해주어야 함.
# 주의할 점은 나는 총 4개의 객체를 탐지해야하기 때문에, 4개의 값을 구해야함.
anchors = [
    # S=13일 때의 scale 값
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
    # S=26일 때의 scale 값
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
    # S=52일 때의 scale 값
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)]]

classes = [ "AC", "FL", "HC", "HUM" ]

In [ ]:
# train.csv 파일 읽어오기
df = pd.read_csv(train_csv)

In [ ]:
df

### transform 정의

In [ ]:
data_transform = transform[(
    
)]

In [ ]:
class YOLODataset(Dataset):
    def __init__(
        self,
        csv_file,
        img_dir,
        label_dir,
        anchors,
        image_size=416,
        S=[13, 26, 52],
        C=4,
        transform=None,
    ):
        self.annotations = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.image_size = image_size
        self.transform = transform
        self.S = S
        
        self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2])  # for all 3 scales
        self.num_anchors = self.anchors.shape[0]
        self.num_anchors_per_scale = self.num_anchors // 3
        self.C = C
        self.ignore_iou_thresh = 0.5

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        label_path = os.path.join(self.label_dir, self.annotations.iloc[index, 1])
        bboxes = np.roll(np.loadtxt(fname=label_path, delimiter=" ", ndmin=2), 4, axis=1).tolist()
        
        img_path = os.path.join(self.img_dir, self.annotations.iloc[index, 0])
        image = np.array(Image.open(img_path).convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=image, bboxes=bboxes)
            image = augmentations["image"]

        # Below assumes 3 scale predictions (as paper) and same num of anchors per scale
        # target에는 grid cell에서 객체의 위치, 크기, 클래스 정보가 저장이 된다.
        targets = [torch.zeros((self.num_anchors // 3, S, S, 6)) for S in self.S]
        
        for box in range(len(df)):
            # box[2:4] -> df['w'], df['h']의미  =>  수정하기.
            # bounding box와 anchor 사이의 IoU계산한 값 저장. 가장 적절한 anchor 인덱스 찾아냄.
            iou_anchors = iou(torch.tensor(box[2:4]), self.anchors)  
            anchor_indices = iou_anchors.argsort(descending=True, dim=0)
            
            class_label = df['class']
            x = df['center_x']
            y = df['center_y']
            width = df['w']
            height = df['h']
            has_anchor = [False] * 3  # each scale should have one anchor
            
            for anchor_idx in anchor_indices:
                # bounding box가 어떤 크기의 grid cell에 해당하는지 결정.
                scale_idx = anchor_idx // self.num_anchors_per_scale
                anchor_on_scale = anchor_idx % self.num_anchors_per_scale
                S = self.S[scale_idx]
                
                # i, j 계산해서 grid cell 위치 얻기.
                i, j = int(S * y), int(S * x)  # which cell
                anchor_taken = targets[scale_idx][anchor_on_scale, i, j, 0]
                
                # 아직 grid cell에 앵커가 할당되지 않거나, 해당 scale에서 anchor를 사용할 수 없는 경우
                # -> 해당 앵커의 bbox 정보 할당하기
                if not anchor_taken and not has_anchor[scale_idx]:
                    targets[scale_idx][anchor_on_scale, i, j, 0] = 1
                    x_cell, y_cell = S * x - j, S * y - i  # both between [0,1]
                    width_cell, height_cell = (
                        width * S,
                        height * S,
                    )  # can be greater than 1 since it's relative to cell
                    box_coordinates = torch.tensor(
                        [x_cell, y_cell, width_cell, height_cell]
                    )
                    targets[scale_idx][anchor_on_scale, i, j, 1:5] = box_coordinates
                    targets[scale_idx][anchor_on_scale, i, j, 5] = int(class_label)
                    has_anchor[scale_idx] = True
                    
                # 할당은 되었지만, IoU가 설정한 임계값보다 큰 경우
                # -> 해당 앵커에 대한 예측 무시.
                elif not anchor_taken and iou_anchors[anchor_idx] > self.ignore_iou_thresh:
                    targets[scale_idx][anchor_on_scale, i, j, 0] = -1  # ignore prediction
        return image, tuple(targets)

In [ ]:
def test():

    dataset = YOLODataset(
        train_csv,
        img_dir,
        label_dir,
        S,
        anchors=anchors,
        transform=transform,
    )
    
    scaled_anchors = torch.tensor(anchors) / (
        1 / torch.tensor(S).unsqueeze(1).unsqueeze(1).repeat(1, 3, 2)
    )
    
    loader = DataLoader(dataset=dataset, batch_size=1, shuffle=True)
    
    for x, y in loader:
        boxes = []

        for i in range(y[0].shape[1]):
            anchor = scaled_anchors[i]
            print(anchor.shape)
            print(y[i].shape)
            boxes += cells_to_bboxes(
                y[i], is_preds=False, S=y[i].shape[2], anchors=anchor
            )[0]
        boxes = nms(boxes, iou_threshold=1, threshold=0.7, box_format="midpoint")
        print(boxes)
        plot_image(x[0].permute(1, 2, 0).to("cpu"), boxes)

In [ ]:
if __name__ == "__main__":
    test()